In [1]:
using Pkg
Pkg.activate("/Users/jiyong/development/Projects/nImageData.jl")
using OpenCV
using CairoMakie, Images, FileIO


  Activating project at `~/development/Projects/nImageData.jl`


In [ ]:
img = OpenCV.imread("cameraman.png", OpenCV.IMREAD_GRAYSCALE)

In [ ]:
# function Mat2Array(img::OpenCV.Mat{T}) where T<:Integer
#     return permutedims(img.data, [3,2,1])
# end


# function Array2Mat(img::Array{T}) where {T<:Real}
#     ll = length(size(img))
#     @assert ll ∈ (2, 3)
#     if ll == 3 
#         return OpenCV.Mat(permutedims(img, [3,2,1]))
#     else 
#         return OpenCV.Mat(permutedims(stack([img, ]), [3,2,1]))
#     end
# end



# """
#     mat2gray(mat::Matrix{<:Real}, range::Union{Nothing, Tuple{Real, Real}} )

# Convert matrix to gray Image of Images.jl. 
# """
# function mat2gray(mat::Matrix{T}, range::Union{Nothing, Tuple{Real, Real}} = nothing ) where T<: Real
#     if range === nothing
#         mv, Mv = extrema(mat)
#         return  Gray.((mat .- mv)./(Mv-mv))
#     else 
#         mv, Mv = minmax(range...)
#         return Gray.(clamp.((mat .-mv)/(Mv-mv), zero(T), one(T)))

#     end
# end

# function mat2gray(mat::Matrix{T}, range::Union{Nothing, Tuple{Real, Real}} = nothing ) where T<: Integer
#     return mat2gray(Float32.(mat), range)
# end

# function sand_pepper_noise(img::Matrix{T}, ratio::Real) where {T<:Integer}
#     @assert 0<ratio<1
#     tm, tM = typemin(T), typemax(T)
#     w, h = size(img)
#     Nnoise = round(Int, w*h*ratio)
#     _x, _y = rand(1:w, Nnoise), rand(1:h, Nnoise)
#     for i in 1:Nnoise 
#         if iseven(i)
#             img[_y[i], _x[i]] = tm
#         else 
#             img[_y[i], _x[i]] = tM
#         end
#     end
#     return img
# end 

In [ ]:
arr1=Mat2Array(img)
arr2 = sand_pepper_noise(arr1[:,:,1], 0.03)

In [ ]:
#save("noisy_cameraman.png", mat2gray(arr2[:,:,1]), )
mat2gray(arr2[:,:,1])

In [ ]:
size(Array2Mat(arr2))

In [ ]:
img3 = OpenCV.medianBlur(Array2Mat(arr2), 5)

In [ ]:
println(size(img3))
mat2gray(Mat2Array(img3)[:,:,1])

In [ ]:
size(Array2Mat(stack([arr2,])))

In [ ]:
x4=OpenCV.medianBlur(Array2Mat(stack([arr2,])),3)[1,:,:]

In [ ]:
mat2gray(x4)

In [ ]:
x4 = OpenCV.medianBlur(Array2Mat())

In [ ]:
b=permutedims(a, [3,2,1])

In [ ]:
permutedims(b, [3,2,1])

In [ ]:
rand(1:100, (3, 4))